# EOAP Generator

One of the Pathfinder delivery partners, Oxidian, has created a tool to help dev-ops specialists or specialist technicians to create compliant EOAPs that will run on the EODH. The eoap-gen tool can be found here: https://github.com/EO-DataHub/eoap-gen  

It describes itself as "a CLI tool for generating Earth Observation Application Packages including CWL workflows and Dockerfiles from user supplied python scripts".

Other tools exist
scriptcwl: https://github.com/NLeSC/scriptcwl and https://scriptcwl.readthedocs.io/en/latest/
cwl-utils: https://github.com/common-workflow-language/cwl-utils and https://cwl-utils.readthedocs.io/en/latest/#





Installation and set up of eoap-gen




https://github.com/EO-DataHub/eoap-gen/blob/main/ades_guide.md